In [1]:
# RUN THIS CELL: it loads some style files
from IPython.core.display import HTML, display, Math
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

# Test binomiale (una coda)

Un'urna contiene monete equilibrate e monete difettose. Le monete equilibrate hanno probabilità di successo $p=1/2$ le monete difettose hanno probabilità di successo ignota $p>1/2$. Non conosciamo la frazione di monete difettose. Nel gergo dei <mark>test di ipotesi</mark> riassumeremo questa situazione scrivendo:

$H_0:\kern3.5ex p=1/2\quad$ <mark>ipotesi nulla</mark>

$H_A:\kern3ex p>1/2\quad$ <mark>ipotesi alternativa</mark>
 
In questo modo dichiariamo che $p=1/2$ è ciò che consideraimo *normale* o *sano* e che  $p>1/2$ è il tipo di deviazione dalla normalità o *patologia* che vogliamo rilevare.

Estraiamo una moneta dall'urna e, per decidere tra equilibrata o difettosa, facciamo il seguente test: la lanciamo $40$ volte e se il numero dei successi è $\ge k$ la dichiariamo difettosa. Stiamo descivendo una famiglia di test, uno per ogni scelta del parametro $k$. Questo parametro lo decide chi progetta il test cercando di minimizzare gli errori. Vogliamo per prima cosa come varia la probabilità di falsi positivi/negativi al variare di $k$. 

Il test è una variabile aleatoria $X$ a valori in $\{0,\dots,40\}$. Lo spazio campionario $\Omega$ è diviso in due parti: $H_A$ e $H_0$.  L'insieme $H_A$ contiene quegli $\omega$ che corrispondono a $40$ lanci fatti con una moneta difettosa mentre $H_0$ contiene quegli $\omega$ che corrispondono a lanci con una moneta equilibrata. 

Condizionando a $H_0$ otteniamo $X\sim {\rm B}(40,\ 1/2)$. Condizionando a $H_A$ otteniamo $X\sim {\rm B}(40,\ p)$ con $p>1/2$ ignota.

In [2]:
from scipy.stats import binom                             # lib. for statistical functions
from ipywidgets import interactive, FloatSlider, IntSlider   # lib. for interactive graphic
from bokeh.io import push_notebook, show, output_notebook # lib. for graphic output
from bokeh.plotting import figure
output_notebook()
options = dict(plot_height=250,plot_width=700,
               tools="pan,wheel_zoom,reset,save,crosshair,box_select")
def printred(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#a02">' + s + '</pre>') )
def printgrn(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#080">' + s + '</pre>') )

Loading BokehJS ...

In [5]:
n = 40; p0 = 1/2; k_max = int(3*n/4)                                # Set the initial values of the parameters
x = range(0,n+1)

plot = figure(title="PMF of X ~ B(n, p) and the interval [k, n].",  # Create an empty figure
              x_axis_label = "# successes", y_axis_label = "probability", 
              x_range=(0,n+1), y_range=(0,max(binom.pmf(x,n,p0)*1.1)), **options )               
plot.title.text_font="courier"
plot.title.text_font_size="14pt"


r1 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='green', alpha=0.5) # Initialize barplot
r2 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='red',   alpha=0.5) # Initialize barplot
show(plot, notebook_handle=True)                                                    # Show barplot
        
def f(k, p, p0):
    data = {'x':x, 'top':  binom.pmf(x, n, p) }
    x_rif = range(k,n+1)
    data2 = {'x':x_rif, 'top':  binom.pmf(x_rif, n, p) }
    r1.data_source.data = data
    r2.data_source.data = data2
    printred('T = [ k, n ]  zona di rifiuto  k = {}  n = {}'.format(k,n) )
    if p == p0: 
        printred('Pr( T | p = {:.2f}) = α = {:.3f} (falso positivo) ERRORE I tipo'.format(p,1-binom.cdf(k-1, n, p)))
    else: 
        printgrn('Pr(¬T | p = {:.2f}) = β = {:.3f} (falso negativo) ERRORE II tipo'.format(p,binom.cdf(k-1, n, p)))
    push_notebook()
    
w1 = interactive(lambda k,p: f(k, p, 0.5),
         k = IntSlider  (description="k", min=n/2, max=k_max, step=1, value=n), 
         p = FloatSlider(description="p", min=0.5, max=0.8, step=0.1, value=0.5))
display(w1);

interactive(children=(IntSlider(value=30, description='k', max=30, min=20), FloatSlider(value=0.5, description…

# Osservazione

$\alpha =\Pr (X\in T\,|\, H_0)$ si chiama <mark>significatività</mark> 

$\displaystyle\alpha =\sum_{k\in T}{n \choose k}\cdot 0.5^k \cdot 0.5^{n-k}$ nell'esempio

$\alpha$ misura la probabilità si un errore del <mark>I tipo</mark> (falso positivo)

$\beta = \Pr (X\notin T\,|\, H_A)$ non ha nome, ma $1-\beta$ si chiama <mark>potenza</mark>, nei test diagnostici la potenza si chiamava *sensitività*. 

$\displaystyle\beta =\sum_{k\in T}{n \choose k}\, p^k \, (1-p)^{n-k}$ nell'esempio

$\beta$ misura la probabilità si un errore del <mark>II tipo</mark> (falso negativo). 

Si noti però che $\beta$ non è ben definito perché varia  al variare di $p$. 
Di solito si prende il massimo possibile.
Il massimo si ottiene quando $p>1/2$ si avvicina a $1/2$.
Quando $p\approx1/2$ allora $\beta\approx 1-\alpha$.
Ma generalmente non siamo interessati a differenze infinitesime. 
Di solito si fissa un <mark>effect size $\delta$</mark> ovvero una minima differenza rilevante.

$\delta = 0.1$ nell'esempio.

# Prevalenza mancinismo 1 (progettazione di un test)

Il $10\%$ è delle persone sono mancine. Vogliamo stabilire se la caratteristica è ereditaria con una significatività $\alpha=1\%$. Il progetto è di selezionare elezioniamo un campione di $40$ persone con almeno un genitore mancino e misuriamo la proporzione di mancini.

Il test di ipotesi che faremo è

$H_0:\kern3.5ex p=0.1\quad$ la prevalenza del mancinismo tra i figli di mancini è invariata

$H_A:\kern3.5ex p>0.1\quad$  la prevalenza del mancinismo tra i figli di mancini è maggiore


In [4]:
show(plot, notebook_handle=True)
w2 = interactive(lambda k,p: f(k, p, 0.1),
         k = IntSlider  (description="k", min=0.1*n/2, max=k_max, step=1, value=n), 
         p = FloatSlider(description="p", min=0.1, max=0.5, step=0.05, value=0.1))
display(w2);

interactive(children=(IntSlider(value=30, description='k', max=30, min=2), FloatSlider(value=0.1, description=…

Nell'esempio qui sopra abbiamo fissato a $0.01=1\%$ la massima probabilità di errore del I tipo che siamo disposti a tollerare. La zona di rifiuto è determinata di conseguenza $[k_{0.01}, n]$.

Il valore $k_{0.01}$ che delimita la zona di rifiuto si chiama <mark>valore critico</mark>. 

È tale che $\Pr(X\ge k_{0.01})\cong 0.01$. (Idealmente vorremmo un $=$, ma per distribuzioni ci dobbiamo accontentare di una approssimazione.)

Quindi un altro modo per dire la stessa cosa è dire che $k_{0.01}$ è quel valore di soglia tale che la probabilità di un errore del I tipo è proprio $0.01$ 

Poiché $1-\Pr(X\le k-1)=\Pr(X\ge k)$.la condizione qui sopra. È equivalente a dire $k_{0.01}$ è tale che $0.99\cong\Pr(X\le k-1)$.
 
La funzione `binom.ppf(q,n,p)` calcola il $k$ take che $q\cong\Pr(X\le k)$ è chiamata funzione <mark>quantile</mark> o anche funzione <mark>percentile</mark>.

Nell'esempio numerico,  $q = 1-\alpha=0.99$.

     qui sotto il calcolo numerico di k_α per α = .01  n = 40  p = .1

In [15]:
n = 40
p = 0.1

for k in range(8,12):
    print ('se k ={:2d} allora Pr(X ≥ k) = 1-binom.cdf(k-1,n,p) = {:.3f}'.format(k,1-binom.cdf(k-1,n,p)))

alpha = 0.01
print ('\nk_α = binom.ppf(1-alpha,n,p) + 1  = {:.0f}'.format(binom.ppf(1-alpha,n,p) + 1))

se k = 8 allora Pr(X ≥ k) = 1-binom.cdf(k-1,n,p) = 0.042
se k = 9 allora Pr(X ≥ k) = 1-binom.cdf(k-1,n,p) = 0.015
se k =10 allora Pr(X ≥ k) = 1-binom.cdf(k-1,n,p) = 0.005
se k =11 allora Pr(X ≥ k) = 1-binom.cdf(k-1,n,p) = 0.001

k_α = binom.ppf(1-alpha,n,p) + 1  = 10


Supponiamo che dal nostro esperimento osserviamo $x$ mancini nel campione di $40$. C'è un modo semplice (senza calcolare $k_\alpha$) per sapere se possiamo rifiutare l'ipotesi nulla con la significatività $\alpha=1\%$. 

Infatti, se $\Pr(X\ge x\,|\,H_0)\le\alpha$ allora sicuramente $k_\alpha<x$.

La probabilità  $\Pr(X\ge x\,|\,H_0)$ si chiama <mark>p-valore</mark>. 

    Un paio di esempi numerici qui sotto.

In [12]:
print('Se x = 9 allora Pr(X ≥ x | H_0) = {:.3f} > α quini non rifiutiamo H0'.format(1 - binom.cdf(8, 40, .1) ) )
print('Se x =11 allora Pr(X ≥ x | H_0) = {:.3f} < α quini     rifiutiamo H0'.format(1 - binom.cdf(9, 40, .1) ) )

Se x = 9 allora Pr(X ≥ x | H_0) = 0.015 > α quini non rifiutiamo H0
Se x =11 allora Pr(X ≥ x | H_0) = 0.005 < α quini     rifiutiamo H0
